# Exploring EM estimates of Fellegi-Sunter parameters from datasets with known parameters

[`splink_data_generation`](https://github.com/moj-analytical-services/splink_data_generation) is a Python package that is able to generate datasets with known parameters.

It has two main ways of generating data:
- `generate_df_gammas_exact`, which produces a dataset with parameters that precisely match the `m` and `u` values specified, and that precisely obeys the assumption of independence of comparison of linking variables conditional on match status.  This function has three limitations:  

    (1) It's not possible to control the overall proportion of matches in the output dataset, and 
    
    (2) It's not possible to generate data that breaks the assumption of conditional independence.
    
    (3) As the complexity of the dataset requested increases, the number of rows generated to satisfy the assumptions can be very high

- `generate_df_gammas_random`, which produces rows at random using the data generating mechanism specified by the supplied parameters.  This function allows the user to specify the proportion of matches, and a covariance matrix which dictates correlations between linking variables conditional on match status.  Since rows are generated at random, the parameters of the resultant dataset will converge to true parameters and the number of rows generated tends to infinity.  



In this notebook, I demonstrate the ability of the EM algorithm to correctly estimate parameters.  I also explore convergence and demonstrate the possibilty that there can sometimes be several local optima, and the EM algorithm doesn't necessarily converge to the parameters of the data generating mechanism.


## Generate the data

We start by using a `splink` settings object (see [editor](https://moj-analytical-services.github.io/splink_settings_editor/)) to specify the columns for our dataframe and the `m` and `u` probabilities.

I have set a few other options to ensure more precise convergnce, and to make sure that the datasets produced by `splink` retain useful information such as the 'ground truth' label of whether the row is a match according to the data generating process. 

In [15]:
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)
from pyspark.sql import Row


In [2]:
%load_ext autoreload
%autoreload 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os 
import sys
sys.path.insert(0, '/Users/robinlinacre/Documents/data_linking/splink_data_generation')

In [98]:
settings = {
    "proportion_of_matches": 0.5,
    "link_type": "dedupe_only",
    "comparison_columns": [
        {
            "col_name": "col_1",
            "m_probabilities": [0.3, 0.7],  # Probability of typo
            "u_probabilities": [0.9, 0.1],  # Probability of collision
        },
        {
            "col_name": "col_2",
            "m_probabilities": [0.1, 0.9],  # Probability of typo
            "u_probabilities": [0.975, 0.025],  # Probability of collision
        },
        {
            "col_name": "col_3",
            "m_probabilities": [0.05, 0.95],  # Probability of typo
            "u_probabilities": [0.8, 0.2],  # Probability of collision
        },
    ],
    "max_iterations": 0,
    "em_convergence": 0.0001,
#      "additional_columns_to_retain": [
#         "true_match", "true_match_probability"
#     ],
 
    
}

In [99]:
from splink import Splink
data = [
    {"unique_id": 1, "col_1":"a", "col_2": "r", "col_3": "x"},
    {"unique_id": 2, "col_1":"a", "col_2": "r", "col_3": "y"},
    {"unique_id": 3, "col_1":"b", "col_2": "s", "col_3": "z"},
    {"unique_id": 4, "col_1":"a", "col_2": "s", "col_3": "z"},
]

df = spark.createDataFrame(Row(**x) for x in data)
df.createOrReplaceTempView("df")
df.show()

+---------+-----+-----+-----+
|unique_id|col_1|col_2|col_3|
+---------+-----+-----+-----+
|        1|    a|    r|    x|
|        2|    a|    r|    y|
|        3|    b|    s|    z|
|        4|    a|    s|    z|
+---------+-----+-----+-----+



In [100]:
from splink import Splink
from splink.expectation_step import _calculate_log_likelihood_df
from splink.params import Model

linker = Splink(settings, df=df, spark=spark)
df_e = linker.manually_apply_fellegi_sunter_weights()
params = Params(settings, spark)
cols = ["match_probability", "log_likelihood", "gamma_col_1", "gamma_col_2", "gamma_col_3"]
df_pd = _calculate_log_likelihood_df(df_e, params, spark).toPandas()
df = df_pd[cols]

In [101]:
df.to_dict(orient="records")

<ipython-input-101-e030273b3aea>:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  df.to_dict(orient="records")


[{'match_probability': 0.9402985074626865,
  'log_likelihood': -4.089357020711062,
  'gamma_col_1': 1,
  'gamma_col_2': 1,
  'gamma_col_3': 0},
 {'match_probability': 0.0021321961620469087,
  'log_likelihood': -1.0448345829876389,
  'gamma_col_1': 0,
  'gamma_col_2': 0,
  'gamma_col_3': 0},
 {'match_probability': 0.04294478527607361,
  'log_likelihood': -3.2002994392952653,
  'gamma_col_1': 1,
  'gamma_col_2': 0,
  'gamma_col_3': 0},
 {'match_probability': 0.0021321961620469087,
  'log_likelihood': -1.0448345829876389,
  'gamma_col_1': 0,
  'gamma_col_2': 0,
  'gamma_col_3': 0},
 {'match_probability': 0.04294478527607361,
  'log_likelihood': -3.2002994392952653,
  'gamma_col_1': 1,
  'gamma_col_2': 0,
  'gamma_col_3': 0},
 {'match_probability': 0.9827586206896551,
  'log_likelihood': -2.036382052219389,
  'gamma_col_1': 0,
  'gamma_col_2': 1,
  'gamma_col_3': 1}]

In [78]:
from splink.blocking import cartesian_block
from splink.settings import complete_settings_dict
from splink.gammas import add_gammas
from splink.iterate import iterate
from splink.params import Model
from splink.expectation_step import _calculate_log_likelihood_df
settings = complete_settings_dict(settings, spark)
df_comparison = cartesian_block(settings, df=df, spark=spark)

df_gammas = add_gammas(df_comparison, settings, spark=spark)


params = Params(settings, spark)

df_e = iterate(
    df_gammas,
    params,
    settings,
    spark,
    compute_ll=True,

)


df_pd = _calculate_log_likelihood_df(df_e, params, spark).toPandas()



/Users/robinlinacre/anaconda3/lib/python3.8/site-packages/splink/case_statements.py:18: UserWarning: The jaro_winkler_sim user definined function is not available in Spark Or you did not pass 'spark' (the SparkSession) into 'Params' Falling back to using levenshtein in the default string comparison functions You can import these functions using the scala-udf-similarity-0.0.7.jar provided with Splink
  warnings.warn(


In [79]:
cols = ["match_probability", "log_likelihood", "gamma_col_1", "gamma_col_2", "gamma_col_3"]
df_pd[cols]

,match_probability,match_probability,log_likelihood,gamma_col_1,gamma_col_2,gamma_col_3
0,0.797468,0.797468,-4.840893,1,1,0
1,0.000534,0.000534,-0.576431,0,0,0
2,0.011094,0.011094,-2.763035,1,0,0
3,0.000534,0.000534,-0.576431,0,0,0
4,0.011094,0.011094,-2.763035,1,0,0
5,0.934426,0.934426,-2.902242,0,1,1


In [51]:
df_e.toPandas()

,match_probability,unique_id_l,unique_id_r,gamma_col_1,gamma_col_2,gamma_col_3
0,0.797468,1,2,1,1,0
1,0.000534,1,3,0,0,0
2,0.011094,1,4,1,0,0
3,0.000534,2,3,0,0,0
4,0.011094,2,4,1,0,0
5,0.934426,3,4,0,1,1


I now use `splink_data_generation` to generate some data according to these `m` and `u` values:

In [5]:
from splink_data_generation.generate_data_exact import generate_df_gammas_exact
from splink_data_generation.match_prob import add_match_prob, add_match_prob_vectorised
# from splink_data_generation.log_likelihood import add_log_likelihood
df = generate_df_gammas_exact(settings)
df.head()

/Users/robinlinacre/Documents/data_linking/splink_data_generation/splink_data_generation/generate_data_exact.py:130: UserWarning: Note that the proportion_of_matches setting is ignored by this generator. Only the m_probabilities and u_probabilities are observed. The proportion of matches in the generated dataset was 0.500
  warnings.warn(


,gamma_col_1,gamma_col_2,gamma_col_3,true_match_l,true_match_r,unique_id_l,unique_id_r
0,0,0,0,1,1,2aea29f8,df288402
1,0,1,0,1,1,b7ff5bf2,75ef2a67
2,0,1,0,1,1,411c6e37,1194ce7f
3,0,1,0,1,1,d215b682,237211bd
4,0,1,0,1,1,f96885e3,83ab7a58


In [6]:
df_1 = add_match_prob(df, settings)
df_1.head()

,gamma_col_1,gamma_col_2,gamma_col_3,true_match_l,true_match_r,unique_id_l,unique_id_r,true_match_probability_l,true_match_probability_r
0,0,0,0,1,1,2aea29f8,df288402,0.002132,0.002132
1,0,1,0,1,1,b7ff5bf2,75ef2a67,0.428571,0.428571
2,0,1,0,1,1,411c6e37,1194ce7f,0.428571,0.428571
3,0,1,0,1,1,d215b682,237211bd,0.428571,0.428571
4,0,1,0,1,1,f96885e3,83ab7a58,0.428571,0.428571


In [10]:
df_2 = add_match_prob_vectorised(df, settings)
df_2.head()

,gamma_col_1,gamma_col_2,gamma_col_3,true_match_l,true_match_r,unique_id_l,unique_id_r,match_probability
0,0,0,0,1,1,2aea29f8,df288402,0.002132
1,0,1,0,1,1,b7ff5bf2,75ef2a67,0.428571
2,0,1,0,1,1,411c6e37,1194ce7f,0.428571
3,0,1,0,1,1,d215b682,237211bd,0.428571
4,0,1,0,1,1,f96885e3,83ab7a58,0.428571


In [12]:
from splink_data_generation.log_likelihood import add_log_likelihood, add_log_likelihood_vectorised
df = add_log_likelihood(df, settings)
cols = [c for c in df.columns if "_r" not in c]
df[cols].head()

,gamma_col_1,gamma_col_2,gamma_col_3,true_match_l,unique_id_l,_p,_m,_u,_b,match_probability,true_log_likelihood_l
0,0,0,0,1,2aea29f8,0.002132,0.05,0.8,0.0625,0.002132,-1.044835
1,0,1,0,1,b7ff5bf2,0.428571,0.05,0.8,0.0625,0.428571,-4.150915
2,0,1,0,1,411c6e37,0.428571,0.05,0.8,0.0625,0.428571,-4.150915
3,0,1,0,1,d215b682,0.428571,0.05,0.8,0.0625,0.428571,-4.150915
4,0,1,0,1,f96885e3,0.428571,0.05,0.8,0.0625,0.428571,-4.150915


In [14]:
df = add_log_likelihood(df, settings)
df

,gamma_col_1,gamma_col_2,gamma_col_3,true_match_l,true_match_r,unique_id_l,unique_id_r,_p,_m,_u,_b,match_probability,true_log_likelihood_l,true_log_likelihood_r
0,0,0,0,1,1,2aea29f8,df288402,0.002132,0.05,0.8,0.0625,0.002132,-1.044835,-1.044835
1,0,1,0,1,1,b7ff5bf2,75ef2a67,0.428571,0.05,0.8,0.0625,0.428571,-4.150915,-4.150915
2,0,1,0,1,1,411c6e37,1194ce7f,0.428571,0.05,0.8,0.0625,0.428571,-4.150915,-4.150915
3,0,1,0,1,1,d215b682,237211bd,0.428571,0.05,0.8,0.0625,0.428571,-4.150915,-4.150915
4,0,1,0,1,1,f96885e3,83ab7a58,0.428571,0.05,0.8,0.0625,0.428571,-4.150915,-4.150915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1,0,1,0,0,25962e86,88a9a3bd,0.773256,0.95,0.2,4.7500,0.773256,-3.146555,-3.146555
1996,1,0,1,0,0,893922f3,46e7f3f5,0.773256,0.95,0.2,4.7500,0.773256,-3.146555,-3.146555
1997,1,0,1,0,0,0e2d1f9c,364aee9e,0.773256,0.95,0.2,4.7500,0.773256,-3.146555,-3.146555
1998,1,0,1,0,0,1ab50969,1c3563cd,0.773256,0.95,0.2,4.7500,0.773256,-3.146555,-3.146555


In [4]:
from IPython.display import display, Markdown
import numpy as np


binary_prop = df["true_match_l"].mean()
prob_prop = df["true_match_probability_l"].mean()
log_likelihood = sum(df["true_log_likelihood_l"])

md = f"""
The number of rows in the simulated dataset is {len(df):,.0f}

The proportion of matches according to `true_match` status is {binary_prop:,.4f}

The expected proportion of matches according to `true_match_probability` status is {prob_prop:,.4f}

The log likelihood of the dataset given the true parameters is {log_likelihood:,.2f}
"""

display(Markdown(md))


The number of rows in the simulated dataset is 4,000

The proportion of matches according to `true_match` status is 0.5000

The expected proportion of matches according to `true_match_probability` status is 0.5000

The log likelihood of the dataset given the true parameters is -6,507.16


We can see there are correlations amongst the values in comparison vectors:

In [21]:
df[["gamma_col_1", "gamma_col_2", "gamma_col_3"]].corr()

,gamma_col_1,gamma_col_2,gamma_col_3
gamma_col_1,1.000000,0.537339,0.464535
gamma_col_2,0.537339,1.000000,0.665635
gamma_col_3,0.464535,0.665635,1.000000


But as expected there are no correlations when conditioning on match status:

In [25]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
df_m = df[df["true_match_l"]==1]
df_m[["gamma_col_1", "gamma_col_2", "gamma_col_3"]].corr()

,gamma_col_1,gamma_col_2,gamma_col_3
gamma_col_1,1.0000,0.0000,-0.0000
gamma_col_2,0.0000,1.0000,0.0000
gamma_col_3,-0.0000,0.0000,1.0000


## Estimate parameters using Splink.

### Test 1: Use starting parameters equal to true parameters


In this experiment, we give Splink starting parameters equal to the correct parameters.  We expect the algorithm to converge immediately

In [5]:
settings_2 = {
    "proportion_of_matches": 0.5,
    "link_type": "dedupe_only",
    "comparison_columns": [
        {
            "col_name": "col_1",
            "m_probabilities": [0.3, 0.7],  
            "u_probabilities": [0.9, 0.1],  
        },
        {
            "col_name": "col_2",
            "m_probabilities": [0.1, 0.9],  
            "u_probabilities": [0.975, 0.025], 
        },
        {
            "col_name": "col_3",
            "m_probabilities": [0.05, 0.95],  
            "u_probabilities": [0.8, 0.2],  
        },
    ],
     "additional_columns_to_retain": [
        "true_match", "true_match_probability"
    ]
}

In [6]:
import logging 
logging.basicConfig()  # Means logs will print in Jupyter Lab

# Set to DEBUG if you want splink to log the SQL statements it's executing under the hood
logging.getLogger("splink").setLevel(logging.INFO)

from pyspark.context import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [7]:
# Now use Splink to estimate the params from the data
from copy import deepcopy
from splink_data_generation.estimate_splink import estimate

df_e, linker = estimate(df, settings_2 ,spark)
df_e.toPandas().head(5)

/Users/robinlinacre/anaconda3/lib/python3.8/site-packages/splink/case_statements.py:17: UserWarning: The jaro_winkler_sim user definined function is not available in Spark Or you did not pass 'spark' (the SparkSession) into 'Params' Falling back to using levenshtein in the default string comparison functions You can import these functions using the scala-udf-similarity-0.0.6.jar provided with Splink
  warnings.warn(f"The jaro_winkler_sim user definined function is not available in Spark "
INFO:splink.expectation_step:Log likelihood for iteration 0:  -6507.157544158125
INFO:splink.iterate:Iteration 0 complete
INFO:splink.params:The maximum change in parameters was 2.384185793236071e-08 for key π_gamma_col_1_prob_dist_non_match_level_0_probability
INFO:splink.iterate:EM algorithm has converged
INFO:splink.expectation_step:Log likelihood for iteration 1:  -6507.157577685738


,match_probability,unique_id_l,unique_id_r,gamma_col_1,prob_gamma_col_1_non_match,prob_gamma_col_1_match,gamma_col_2,prob_gamma_col_2_non_match,prob_gamma_col_2_match,gamma_col_3,prob_gamma_col_3_non_match,prob_gamma_col_3_match,true_match_l,true_match_r,true_match_probability_l,true_match_probability_r
0,0.002132,b3bbf501,de3c2715,0,0.9,0.3,0,0.975,0.1,0,0.8,0.05,1,1,0.002132,0.002132
1,0.428571,07874d3d,aed91f0f,0,0.9,0.3,1,0.025,0.9,0,0.8,0.05,1,1,0.428571,0.428571
2,0.428571,9d723585,f2ffbdea,0,0.9,0.3,1,0.025,0.9,0,0.8,0.05,1,1,0.428571,0.428571
3,0.428571,265b8dce,9b56092c,0,0.9,0.3,1,0.025,0.9,0,0.8,0.05,1,1,0.428571,0.428571
4,0.428571,0cd9d9d5,30495cc8,0,0.9,0.3,1,0.025,0.9,0,0.8,0.05,1,1,0.428571,0.428571


In [8]:
linker.model

λ (proportion of matches) = 0.5
------------------------------------
gamma_col_1: Comparison of col_1

Probability distribution of gamma values amongst matches:
    value 0: 0.300000 (level represents lowest category of string similarity)
    value 1: 0.700000 (level represents highest category of string similarity)

Probability distribution of gamma values amongst non-matches:
    value 0: 0.900000 (level represents lowest category of string similarity)
    value 1: 0.100000 (level represents highest category of string similarity)
------------------------------------
gamma_col_2: Comparison of col_2

Probability distribution of gamma values amongst matches:
    value 0: 0.100000 (level represents lowest category of string similarity)
    value 1: 0.900000 (level represents highest category of string similarity)

Probability distribution of gamma values amongst non-matches:
    value 0: 0.975000 (level represents lowest category of string similarity)
    value 1: 0.025000 (level repres

As expected, Splink does not iterate away from the correct answer.  

We can see that the EM algorithm does not iterate away from the true parameters, and the log likeilhood of the estimated model is equal to the true log likelihood

What happens if we estimate the model using a different set of starting parameters?

### Test 2: Does Splink correctly estimate parameters with default starting parameters

In [9]:
from copy import deepcopy
settings_3 = {
    "link_type": "dedupe_only",
    "comparison_columns": [
        {
            "col_name": "col_1"
        },
        {
            "col_name": "col_2"
        },
        {
            "col_name": "col_3" 
        },
    ],
    "max_iterations": 200,
    "em_convergence": 0.0001,
     "additional_columns_to_retain": [
        "true_match", "true_match_probability"
    ]
}

settings_copy = deepcopy(settings_3)
df_e, linker = estimate(df, settings_3, spark)
linker.model

/Users/robinlinacre/anaconda3/lib/python3.8/site-packages/splink/case_statements.py:17: UserWarning: The jaro_winkler_sim user definined function is not available in Spark Or you did not pass 'spark' (the SparkSession) into 'Params' Falling back to using levenshtein in the default string comparison functions You can import these functions using the scala-udf-similarity-0.0.6.jar provided with Splink
  warnings.warn(f"The jaro_winkler_sim user definined function is not available in Spark "
INFO:splink.expectation_step:Log likelihood for iteration 0:  -7206.433293015898
INFO:splink.iterate:Iteration 0 complete
INFO:splink.params:The maximum change in parameters was 0.14686706066131594 for key π_gamma_col_3_prob_dist_non_match_level_0_probability
INFO:splink.expectation_step:Log likelihood for iteration 1:  -6552.875324091865
INFO:splink.iterate:Iteration 1 complete
INFO:splink.params:The maximum change in parameters was 0.03220529854297638 for key π_gamma_col_1_prob_dist_match_level_0_pr

λ (proportion of matches) = 0.4996649920940399
------------------------------------
gamma_col_1: Comparison of col_1

Probability distribution of gamma values amongst matches:
    value 0: 0.299742 (level represents lowest category of string similarity)
    value 1: 0.700258 (level represents highest category of string similarity)

Probability distribution of gamma values amongst non-matches:
    value 0: 0.899855 (level represents lowest category of string similarity)
    value 1: 0.100144 (level represents highest category of string similarity)
------------------------------------
gamma_col_2: Comparison of col_2

Probability distribution of gamma values amongst matches:
    value 0: 0.099721 (level represents lowest category of string similarity)
    value 1: 0.900279 (level represents highest category of string similarity)

Probability distribution of gamma values amongst non-matches:
    value 0: 0.974693 (level represents lowest category of string similarity)
    value 1: 0.02530

In [10]:
from splink.expectation_step import _calculate_log_likelihood_df
df_3 = _calculate_log_likelihood_df(df_e, linker.model, spark).toPandas()
df_3["log_likelihood"].sum()


-6507.159182916064

In [1]:
import os 
import sys
sys.path.insert(0, '/Users/robinlinacre/Documents/data_linking/splink_data_generation')

from splink_data_generation.generate_data_exact import generate_df_gammas_exact
from splink_data_generation.match_prob import add_match_prob, add_match_prob_vectorised
from splink_data_generation.log_likelihood import (
add_log_likelihood,
add_log_likelihood_vectorised,
)
import pandas as pd

import numpy as np


settings = {
    "proportion_of_matches": 0.5,
    "link_type": "dedupe_only",
    "comparison_columns": [
        {
            "col_name": "col_1",
            "m_probabilities": [0.3, 0.7],  # Probability of typo
            "u_probabilities": [0.9, 0.1],  # Probability of collision
        },
        {
            "col_name": "col_2",
            "m_probabilities": [0.1, 0.9],  # Probability of typo
            "u_probabilities": [0.975, 0.025],  # Probability of collision
        },
        {
            "col_name": "col_3",
            "m_probabilities": [0.05, 0.95],  # Probability of typo
            "u_probabilities": [0.8, 0.2],  # Probability of collision
        },
    ],
    "max_iterations": 0,
    "em_convergence": 0.0001,
}

expected_result_rows = [{'match_probability': 0.9402985074626865,
  'log_likelihood': -4.089357020711062,
  'gamma_col_1': 1,
  'gamma_col_2': 1,
  'gamma_col_3': 0},
 {'match_probability': 0.0021321961620469087,
  'log_likelihood': -1.0448345829876389,
  'gamma_col_1': 0,
  'gamma_col_2': 0,
  'gamma_col_3': 0},
 {'match_probability': 0.04294478527607361,
  'log_likelihood': -3.2002994392952653,
  'gamma_col_1': 1,
  'gamma_col_2': 0,
  'gamma_col_3': 0},
 {'match_probability': 0.0021321961620469087,
  'log_likelihood': -1.0448345829876389,
  'gamma_col_1': 0,
  'gamma_col_2': 0,
  'gamma_col_3': 0},
 {'match_probability': 0.04294478527607361,
  'log_likelihood': -3.2002994392952653,
  'gamma_col_1': 1,
  'gamma_col_2': 0,
  'gamma_col_3': 0},
 {'match_probability': 0.9827586206896551,
  'log_likelihood': -2.036382052219389,
  'gamma_col_1': 0,
  'gamma_col_2': 1,
  'gamma_col_3': 1}]
df_expected = pd.DataFrame(expected_result_rows)

df = generate_df_gammas_exact(settings)
df = add_match_prob(df, settings)
df = add_log_likelihood_vectorised(df, settings)

join = ["gamma_col_1", "gamma_col_2", "gamma_col_3"]
merged = df.merge(df_expected, left_on=join, right_on=join, how="inner").drop_duplicates(join)
merged


/Users/robinlinacre/Documents/data_linking/splink_data_generation/splink_data_generation/generate_data_exact.py:130: UserWarning: Note that the proportion_of_matches setting is ignored by this generator. Only the m_probabilities and u_probabilities are observed. The proportion of matches in the generated dataset was 0.500
  warnings.warn(


,gamma_col_1,gamma_col_2,gamma_col_3,true_match_l,true_match_r,unique_id_l,unique_id_r,true_match_probability_l,true_match_probability_r,true_log_likelihood_l,true_log_likelihood_r,match_probability,log_likelihood
0,0,0,0,1,1,799eb687,66779231,0.002132,0.002132,-1.044835,-1.044835,0.002132,-1.044835
2814,1,0,0,1,1,2e5207cd,655153c6,0.042945,0.042945,-3.200299,-3.200299,0.042945,-3.200299
3140,1,1,0,1,1,937abefe,6acb7a55,0.940299,0.940299,-4.089357,-4.089357,0.940299,-4.089357
3207,0,1,1,1,1,c25f3b15,bdb5e23b,0.982759,0.982759,-2.036382,-2.036382,0.982759,-2.036382


In [103]:
df.head()

,gamma_col_1,gamma_col_2,gamma_col_3,true_match_l,true_match_r,unique_id_l,unique_id_r,true_match_probability_l,true_match_probability_r,true_log_likelihood_l,true_log_likelihood_r
0,0,0,0,1,1,2fb5db19,17c23fc9,0.002132,0.002132,0.0015,0.0015
1,0,1,0,1,1,fd51e2b9,62a4f45c,0.428571,0.428571,0.0135,0.0135
2,0,1,0,1,1,c0105d87,0c3378ab,0.428571,0.428571,0.0135,0.0135
3,0,1,0,1,1,6a11139a,cf583cc5,0.428571,0.428571,0.0135,0.0135
4,0,1,0,1,1,3d05b757,0aadebac,0.428571,0.428571,0.0135,0.0135


In [104]:
df_expected

,match_probability,log_likelihood,gamma_col_1,gamma_col_2,gamma_col_3
0,0.940299,-4.089357,1,1,0
1,0.002132,-1.044835,0,0,0
2,0.042945,-3.200299,1,0,0
3,0.002132,-1.044835,0,0,0
4,0.042945,-3.200299,1,0,0
5,0.982759,-2.036382,0,1,1


In [94]:
join = ["gamma_col_1", "gamma_col_2", "gamma_col_3"]
df.merge(df_expected, left_on=join, right_on=join, how="inner").drop_duplicates(join)

,gamma_col_1,gamma_col_2,gamma_col_3,true_match_l,true_match_r,unique_id_l,unique_id_r,true_match_probability_l,true_match_probability_r,true_log_likelihood_l,true_log_likelihood_r,match_probability,log_likelihood
0,0,0,0,1,1,9f851072,c21a4293,0.002132,0.002132,0.0015,0.0015,0.000534,-0.576431
2814,1,0,0,1,1,0b2175ac,bf5b261b,0.042945,0.042945,0.0035,0.0035,0.011094,-2.763035
3140,1,1,0,1,1,871253fc,d46dd6a8,0.940299,0.940299,0.0315,0.0315,0.797468,-4.840893
3207,0,1,1,1,1,ce8ba011,e1a1b460,0.982759,0.982759,0.2565,0.2565,0.934426,-2.902242
